In [2]:
from aic_ml.MAHIL.helper.utils import load_trajectories

data_dir = "/home/sseo/Projects/ai_coach/train_ma_dnn/data/"
# file_name = "Terran5v5_100.pkl"
file_name = "Terran5v5_100.pkl"
traj = load_trajectories(data_dir + file_name, 100, 0)

In [3]:
import numpy as np

n_epi = len(traj[0]["rewards"])
n_agent = len(traj)
list_epi_rews = [list() for _ in range(n_agent)]
list_wons = []
for i_e in range(n_epi):
    if "wons" in traj[0]:
        list_wons.append(traj[0]["wons"][i_e])
    for i_a in range(n_agent):
        epi_rew = np.sum(traj[i_a]["rewards"][i_e])
        list_epi_rews[i_a].append(epi_rew)

print(np.mean(list_epi_rews, axis=1))
print(np.mean(list_wons))
    

[11.85184069 11.85184069 11.85184069 11.85184069 11.85184069]
0.6


0
